In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
def prepare_data():
    data = np.load('/content/gdrive/My Drive/data/stereo_matching/cosine_1.npy')
    
    data = data.transpose((0, 4, 2, 3, 1))
    data = data.reshape(data.shape[0], 15, 15, 2)
    
    # normalization
    mean = np.mean(data)
    std  = np.std(data)
    data = (data - mean)/(std+1e-7)
    
    return data

In [3]:
# get data
data = prepare_data()
print(data.shape)

(1000000, 15, 15, 2)


In [4]:
# get labels
Y = np.load('/content/gdrive/My Drive/data/stereo_matching/labels_consine.npy')
print(Y.shape)

(1000000,)


In [5]:
# shuffle
data_size = data.shape[0]

rng = np.random.RandomState(113)
indices = np.arange(data_size)

rng.shuffle(indices)
data = data[indices]
Y = Y[indices]

In [6]:
BATCH_SIZE = 512
SHUFFLE_BUFFER_SIZE = 1024

train_dataset = tf.data.Dataset.from_tensor_slices((data, Y))
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

for X1,Y1 in train_dataset.take(1):
    print(X1.shape)
    print(Y1.shape)

(512, 15, 15, 2)
(512,)


In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization
import tensorflow as tf

def base_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(8, (1,1),activation='relu', kernel_initializer='he_uniform')(inputs)
    x = Conv2D(8, (3,3), activation='relu', kernel_initializer='he_uniform')(x)
    x = Conv2D(8, (3,3), activation='relu', kernel_initializer='he_uniform')(x)
    x = Conv2D(8, (3,3), activation='relu', kernel_initializer='he_uniform')(x)
    x = Conv2D(8, (3,3), activation='relu', kernel_initializer='he_uniform')(x)
    x = Conv2D(16, (3,3), activation='relu', kernel_initializer='he_uniform')(x)
    x = Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform')(x)
    x = Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform')(x)
    x = Conv2D(1, (1,1), activation='tanh')(x)
    x = tf.squeeze(x)
    model = Model(inputs=inputs, outputs=x)
    return model

In [8]:
model = base_model((None,None,2))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 2)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 8)     24        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 8)     584       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 8)     584       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 8)     584       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 8)     584       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 16)    1168  

In [9]:
X = tf.random.normal((128, 15, 15, 2))
Y = tf.random.normal((128,))

print(X.shape)
print(Y.shape)

(128, 15, 15, 2)
(128,)


In [10]:
# predict
pred = model(X)

# loss
print(pred.shape)
print(Y.shape)

(128,)
(128,)


In [11]:
optimizer = tf.keras.optimizers.Adam()

epochs = 30
for i in range(epochs):
  # monitor
  loss_mean = []
  for X,Y in train_dataset:
    with tf.GradientTape() as tape:
      output = model(X)
      loss   = tf.keras.losses.MSE(Y, output)
      loss_mean.append(loss.numpy().mean())
    gradient = tape.gradient(loss, model.trainable_weights)     
    optimizer.apply_gradients(zip(gradient, model.trainable_weights))
      
  print('\nLoss at epoch ', i, ' with loss=', sum(loss_mean)/len(loss_mean))


Loss at epoch  0  with loss= 0.018192377085621138

Loss at epoch  1  with loss= 0.005549663940907867

Loss at epoch  2  with loss= 0.003935034517646827

Loss at epoch  3  with loss= 0.0032166015462112923

Loss at epoch  4  with loss= 0.002844252840719713

Loss at epoch  5  with loss= 0.0025694701815130784

Loss at epoch  6  with loss= 0.0023450705235535155

Loss at epoch  7  with loss= 0.0022110200464795083

Loss at epoch  8  with loss= 0.002105641832833532

Loss at epoch  9  with loss= 0.0020120935975106867

Loss at epoch  10  with loss= 0.0019109294014723595

Loss at epoch  11  with loss= 0.001849442498366174

Loss at epoch  12  with loss= 0.0017782320806917714

Loss at epoch  13  with loss= 0.0017430349634416943

Loss at epoch  14  with loss= 0.001681169424088769

Loss at epoch  15  with loss= 0.0016500213908049695

Loss at epoch  16  with loss= 0.0016020625473301335

Loss at epoch  17  with loss= 0.0015721930574722008

Loss at epoch  18  with loss= 0.0015324617504521617

Loss at e

In [12]:
#Save model to cosin.h5 file
model.save('/content/gdrive/My Drive/data/stereo_matching/model_cosine_tape_v1.h5')